In [1]:
with open('input.txt', "r") as f:
    lines = f.read().rstrip().split('\n')
lines = list(map(lambda x: x.replace(" ", ""), lines))
lines[:5]

['9+4+(3+(6*5+9*2*4)*9)',
 '((7+6)+2*6+4*2+9)*6+5',
 '7*3*9*6+(4+2+4*9)+7',
 '9+(9*3*3*(9+5+4)+4)',
 '(8*5+(7+5+3+6)*9+7*(9*8+7*2))*8']

In [2]:
##Copy-paste from https://stackoverflow.com/questions/29991917/indices-of-matching-parentheses-in-python/29992065

def find_matching_parentheses(text):
    istart = []  # stack of indices of opening parentheses
    d = {}

    for i, c in enumerate(text):
        if c == '(':
             istart.append(i)
        if c == ')':
            try:
                d[istart.pop()] = i
            except IndexError:
                print('Too many closing parentheses')
    if istart:  # check if stack is empty afterwards
        print('Too many opening parentheses')
    return d

print(lines[1])
find_matching_parentheses(lines[1])

((7+6)+2*6+4*2+9)*6+5


{1: 5, 0: 16}

In [3]:
def calc_num(x, y, op):
    if op == "+":
        return str(int(x) + int(y))
    elif op == "*":
        return str(int(x) * int(y))
        

In [5]:
import re
from collections import deque

def calc_sequential(text):
    nums = deque(re.findall("\d+", text))
    operators = deque(re.findall("[+|*]",text))
    x = nums[0]
    nums.popleft()
    calc = x
    while(len(nums) > 0):
        y = nums[0]
        op = operators[0]
        eval_string = f"{calc}{op}{y}"
        calc = str(eval(eval_string))
        nums.popleft()
        operators.popleft()
    return calc
                     
calc_sequential("1 + 2 * 3 + 4 * 5 + 6".replace(' ', ''))                

'71'

In [20]:
%%time
import re

def remove_summation(text):
    if "+" not in text:
        return text
    
    last_y = re.findall("\d+", text)[-1]
    new_eq = text
    
    i = 0
    while(i < len(text)-len(last_y)):
        x = re.findall("\d+", text[i:])[0]
        y = re.findall("\d+", text[i+len(x)+1:])[0]
        op = re.findall("[+|*]",text[i+len(x):])[0]
        
        if op =="+":
            new_eq = new_eq.replace(f"{x}+{y}", calc_num(x, y, op), 1)
            return remove_summation(new_eq)
        else:
            i += 1+len(x)


def calculate(text, reverse_priority):
    new_eq = text
    x = re.findall("\d+", text)[0]
    calc = x
    last_y = re.findall("\d+", text)[-1]
    
    if "(" not in text:
        if reverse_priority:
            text = remove_summation(text)
            calc = eval(text)
        else:
            calc = calc_sequential(text)
        return calc
    else: 
        while (True):
            if "(" not in new_eq:
                break

            else:
                match_pars = list(find_matching_parentheses(new_eq).items())[0]
                start_par_idx = match_pars[0]
                close_par_idx = match_pars[1]
                substr = new_eq[start_par_idx:close_par_idx+1]
                new_eq = new_eq.replace(substr, str(calculate(substr[1:-1], reverse_priority)))
                
        return calculate(new_eq, reverse_priority)
    return calc


calcs = list(map(int, [calculate(x, False) for x in lines]))
sum(calcs)

CPU times: user 50.5 ms, sys: 1.41 ms, total: 51.9 ms
Wall time: 50.9 ms


24650385570008

In [19]:
%%time
#Part 2

calcs = list(map(int, [calculate(x, True) for x in lines]))
sum(calcs)

CPU times: user 53.2 ms, sys: 1.28 ms, total: 54.4 ms
Wall time: 53.5 ms


158183007916215